In [7]:
import numpy as np
import matplotlib.pyplot as plt


# 设置 LaTeX 样式
plt.rc('text', usetex=True)
plt.rc('font', family='serif')


# 定义时间范围
Time = np.linspace(1173, 2025, 1000)
dt = Time[1] - Time[0]

# 定义年度游客人数模型
def annual_visitors_updated(t):
    def fluctuation(t, A, T):
        return 1 + A * np.sin(2 * np.pi * t / T)

    if t < 1372:
        N0 = 100
        r0 = 1
        A = 0.05
        T = 10
        return (N0 + r0 * (t - 1173)) * fluctuation(t, A, T)
    elif t < 1838:
        N1 = 2000
        r1 = 0.01
        A = 0.1
        T = 20
        return N1 * np.exp(r1 * (t - 1372)) * fluctuation(t, A, T)
    elif t < 1934:
        N2 = 5000
        r2 = 0.005
        A = 0.2
        T = 15
        return N2 * np.exp(r2 * (t - 1838)) * fluctuation(t, A, T)
    elif t < 1990:
        N3 = 100000
        r3 = 0.02
        A = 0.15
        T = 5
        return N3 * np.exp(r3 * (t - 1990)) * fluctuation(t, A, T)
    elif t < 2001:
        N4 = 100
        r4 = 1
        A = 0.05
        T = 10
        return (N4 + r4 * (t - 1173)) * fluctuation(t, A, T)
    elif t < 2020:
        N5 = 300000
        r5 = 0.07
        A = 0.05
        T = 3
        return N5 * np.exp(r5 * (t - 2001)) * fluctuation(t, A, T)
    elif t < 2022:
        N6 = 120000
        return N6
    else:
        N7 = 600000
        r5 = 0.05
        A = 0.1
        T = 5
        return N7 * np.exp(r5 * (t - 2022)) * fluctuation(t, A, T)

# 计算年度游客人数
visitor_counts_updated = [annual_visitors_updated(t) for t in Time]

# 摩擦系数动态方程（适用于大理石，石灰岩和砂岩）
def mu_t(t, material="marble", alpha=0.00000001, mu0=0.7, mu_min=0.3):
    # 不同材料的参数调整
    if material == "marble":
        mu0 = 0.7
        mu_min = 0.4
        alpha=0.00000005
    elif material == "limestone":
        mu0 = 0.65
        mu_min = 0.35
        alpha=0.00000003
    elif material == "sandstone":
        mu0 = 0.55
        mu_min = 0.25
        alpha=0.00000001
    
    visitors = np.array([annual_visitors_updated(year) for year in np.linspace(1173, t, 100)])
    cumulative_visitors = np.trapz(visitors, dx=(t - 1173) / len(visitors))
    mu = max(mu_min, mu0 * np.exp(-alpha * cumulative_visitors))
    return mu

# 修复摩擦系数动态方程（适用于大理石，石灰岩和砂岩）
def mu_repair(t, material="marble", mu0=0.7, mu_last=0.3, recovery_factor=0.05, recovery_ratio=0.03):
    # 根据不同材料调整修复系数
    if material == "marble":
        mu0 = 0.7
        recovery_factor=0.1
        recovery_ratio=0.06
    elif material == "limestone":
        mu0 = 0.65
        recovery_factor=0.06
        recovery_ratio=0.04
    elif material == "sandstone":
        mu0 = 0.55
        recovery_factor=0.04
        recovery_ratio=0.02
    
    if 1838 <= t <= 1934:
        return  recovery_ratio * (mu0) * (1 - np.exp(-recovery_factor * (t - 1838)))
    elif 1990 <= t <= 2001:
        return  recovery_ratio * (mu0) * (1 - np.exp(-recovery_factor * (t - 1990)))
    else:
        return 0

# 总摩擦系数（适用于大理石，石灰岩和砂岩）
def total_mu(t, material="marble"):
    mu_non_repair = mu_t(t, material=material)
    mu_repair_phase = mu_repair(t, material=material)
    return mu_non_repair + mu_repair_phase

# 磨损系数动态方程（适用于大理石，石灰岩和砂岩）
def K_t(t, material="marble", beta=0.0000002, K0=1e-3, K_max=1e-2):
    # 根据不同材料调整磨损系数
    if material == "marble":
        K0 = 1e-3
        K_max = 1e-2
        beta=0.0000001
    elif material == "limestone":
        K0 = 8e-4
        K_max = 8e-3
        beta=0.00000002
    elif material == "sandstone":
        K0 = 1.2e-3
        K_max = 1.5e-2
        beta=0.00000004
    
    visitors = np.array([annual_visitors_updated(year) for year in np.linspace(1173, t, 100)])
    cumulative_visitors = np.trapz(visitors, dx=(t - 1173) / len(visitors))
    return min(K_max, K0 * (1 + beta * cumulative_visitors))

# 修复磨损系数动态方程（适用于大理石，石灰岩和砂岩）
def K_repair(t, material="marble", K0=1e-3, K_max=1e-2, recovery_factor=0.05, recovery_ratio=0.02):
    # 根据不同材料调整磨损系数
    if material == "marble":
        K0 = 1e-3
        recovery_factor=0.4
        recovery_ratio=0.24
    elif material == "limestone":
        K0 = 8e-4
        recovery_factor=0.24
        recovery_ratio=0.16
    elif material == "sandstone":
        K0 = 1.2e-3
        recovery_factor=0.16
        recovery_ratio=0.08
    
    if 1838 <= t <= 1934:
        return  -recovery_ratio * (K0) * (1 - np.exp(-recovery_factor * (t - 1838)))
    elif 1990 <= t <= 2001:
        return  -recovery_ratio * (K0) * (1 - np.exp(-recovery_factor * (t - 1990)))
    else:
        return 0  # 非修复期间磨损系数为0

# 总磨损系数（适用于大理石，石灰岩和砂岩）
def total_K(t, material="marble"):
    K_non_repair = K_t(t, material=material)
    K_repair_phase = K_repair(t, material=material)
    return K_non_repair + K_repair_phase

# 硬度动态方程（适用于大理石，石灰岩和砂岩）
def H_t(t, material="marble", gamma=0.000000003, H0=3.5, H_min=3):
    # 根据不同材料调整硬度
    if material == "marble":
        H0 = 3.51
        H_min = 3
        gamma=0.000000005
    elif material == "limestone":
        H0 = 3.5
        H_min = 2.8
        gamma=0.000000003
  
    
    visitors = np.array([annual_visitors_updated(year) for year in np.linspace(1173, t, 100)])
    cumulative_visitors = np.trapz(visitors, dx=(t - 1173) / len(visitors))
    return max(H_min, H0 - gamma * cumulative_visitors)

# 修复硬度动态方程（适用于大理石，石灰岩和砂岩）
def H_repair(t, material="marble", H0=3.5, H_min=3, recovery_factor=0.5, recovery_ratio=0.01):
    # 根据不同材料调整硬度
    if material == "marble":
        H0 = 3.51
        recovery_factor=0.025
        recovery_ratio=0.015
    elif material == "limestone":
        H0 = 3.5
        recovery_factor=0.015
        recovery_ratio=0.01
    
    
    if 1838 <= t <= 1934:
        return recovery_ratio * (H0) * (1 - np.exp(-recovery_factor * (t - 1838)))
    elif 1990 <= t <= 2001:
        return recovery_ratio * (H0) * (1 - np.exp(-recovery_factor * (t - 1990)))
    else:
        return 0

# 总硬度（适用于大理石，石灰岩和砂岩）
def total_H(t, material="marble"):
    H_non_repair = H_t(t, material=material)
    H_repair_phase = H_repair(t, material=material)
    return H_non_repair + H_repair_phase


# 计算不同材料的摩擦系数、磨损系数和硬度
mu_dynamic_marble = [total_mu(t, material="marble") for t in Time]
mu_dynamic_limestone = [total_mu(t, material="limestone") for t in Time]
mu_dynamic_sandstone = [total_mu(t, material="sandstone") for t in Time]

K_dynamic_marble = [total_K(t, material="marble") for t in Time]
K_dynamic_limestone = [total_K(t, material="limestone") for t in Time]
K_dynamic_sandstone = [total_K(t, material="sandstone") for t in Time]

H_dynamic_marble = [total_H(t, material="marble") for t in Time]
H_dynamic_limestone = [total_H(t, material="limestone") for t in Time]



# 绘图
fig, axes = plt.subplots(3, 1, figsize=(14, 12))


# 设置坐标轴样式（像尺子的刻度）
for ax in axes:
    # 调整主刻度和次刻度的间隔
    ax.xaxis.set_major_locator(plt.MaxNLocator(10))  # 主刻度设置为最多10个
    ax.xaxis.set_minor_locator(plt.MaxNLocator(20))  # 次刻度设置为最多20个
    ax.yaxis.set_major_locator(plt.MaxNLocator(10))  # 主刻度设置为最多10个
    ax.yaxis.set_minor_locator(plt.MaxNLocator(20))  # 次刻度设置为最多20个
    
    ax.tick_params(axis='both', which='major', direction='in', length=10, width=1.5, color='black')  # 主刻度
    ax.tick_params(axis='both', which='minor', direction='in', length=5, width=1, color='black')  # 次刻度
    ax.minorticks_on()  # 开启次刻度

    # 仅设置主刻度的网格为虚线，次刻度不添加网格
    ax.grid(visible=True, which='major', linestyle='--', linewidth=0.7, color='lightgrey')
    ax.grid(visible=False, which='minor')  # 关闭次刻度的网格线

# 摩擦系数
axes[0].plot(Time, mu_dynamic_marble, label="Marble", color='orange', linewidth=2)
axes[0].plot(Time, mu_dynamic_limestone, label="Limestone", color='blue', linewidth=2)
axes[0].plot(Time, mu_dynamic_sandstone, label="Sandstone", color='green', linewidth=2)
axes[0].set_title('Dynamic Change of Friction Coefficient Over Time')
axes[0].set_ylabel('Friction Coefficient (μ)')
axes[0].set_xlabel('Year')
axes[0].legend() 
axes[0].grid(True)



# 磨损系数
axes[1].plot(Time, K_dynamic_marble, label="Marble", color='orange', linewidth=2)
axes[1].plot(Time, K_dynamic_limestone, label="Limestone", color='blue', linewidth=2)
axes[1].plot(Time, K_dynamic_sandstone, label="Sandstone", color='green', linewidth=2)
axes[1].set_title('Dynamic Change of Wear Coefficient Over Time')
axes[1].set_ylabel('Wear Coefficient (K)')
axes[1].set_xlabel('Year')
axes[1].legend() 
axes[1].grid(True)


# 硬度
axes[2].plot(Time, H_dynamic_marble, label="Marble", color='orange', linewidth=2)
axes[2].plot(Time, H_dynamic_limestone, label="Limestone", color='blue', linewidth=2)
axes[2].set_title('Dynamic Change of Hardness Over Time')
axes[2].set_ylabel('Hardness (H)')
axes[2].set_xlabel('Year')
axes[2].legend() 
axes[2].grid(True)
plt.legend()
plt.tight_layout()
plt.show()


# 设置 y 轴范围，拉大 y 轴
axes[0].set_ylim(0.3, 0.8)  # 调整摩擦系数的 y 轴范围
axes[1].set_ylim(0, 0.006)  # 调整磨损系数的 y 轴范围
axes[2].set_ylim(3.25, 3.55 )  # 调整硬度的 y 轴范围


# 计算每层楼梯的磨损速率
def wear_rate_layer(t, layer="bottom"):
    n_bottom, n_middle, n_top = 150,  100,  44  # 假定阶数，您可以根据需要调整
    build_time_bottom, build_time_middle, build_time_top = 1173, 1372, 1399  # 每层的建造时间（假定）
    
    if layer == "bottom":
        n = n_bottom
        A_build = (t - build_time_bottom) / 10000  # 假定建造时间影响因子，随时间线性变化
        if t < build_time_bottom:
            return 0  # 如果楼层尚未建成，则磨损速率为0
    elif layer == "middle":
        n = n_middle
        A_build = (t - build_time_middle) / 10000
        if t < build_time_middle:
            return 0  # 如果楼层尚未建成，则磨损速率为0
    else:
        n = n_top
        A_build = (t - build_time_top) / 10000
        if t < build_time_top:
            return 0  # 如果楼层尚未建成，则磨损速率为0

    # 动态参数，基于大理石的相关数据，考虑建造时间的影响
    K = 1e-3 + 1e-5 * t * A_build  # 磨损系数随时间和建造时间增加
    mu = 0.7 - 0.001 * t * A_build  # 摩擦系数随时间和建造时间减少
    H = 3.51 - 0.0001 * t * A_build  # 硬度随时间和建造时间减少
    
    # 修复系数模型，修复加速作用
    def R(t):
        # 修复阶段 1：1838到1934
        if 1838 <= t <= 1934:
            return 1 / np.exp(0.05 * (t - 1838))  # 指数衰减，随时间推移修复效果逐渐减弱
        # 修复阶段 2：1990到2001
        elif 1990 <= t <= 2001:
            return 1 / np.exp(0.05 * (t - 1990))  # 第二次修复，指数衰减
        else:
            return 1  # 非修复时间段，修复系数为1（即无影响）

    # 计算修复系数随时间变化
    R_values = [R(t) for t in Time]


    # 计算磨损速率
    return K * mu * (visitor_counts_updated[int(t) - 1173] / H) * n * A_build * R(t)


# 计算每一层楼梯的磨损速率并绘制图
wear_rates_bottom = [wear_rate_layer(t, layer="bottom") for t in Time]
wear_rates_middle = [wear_rate_layer(t, layer="middle") for t in Time]
wear_rates_top = [wear_rate_layer(t, layer="top") for t in Time]

# 绘制每层楼梯的磨损速率图
plt.figure(figsize=(14, 6))
plt.plot(Time, wear_rates_bottom, label="Bottom Layer Wear Rate", color='orange', linewidth=2)
plt.plot(Time, wear_rates_middle, label="Middle Layer Wear Rate", color='blue', linewidth=2)
plt.plot(Time, wear_rates_top, label="Top Layer Wear Rate", color='green', linewidth=2)

plt.title('Wear Rate of Stairs Over Time (With A and R)', fontsize=16)
plt.xlabel('Year', fontsize=14)
plt.ylabel('Wear Rate (cm³/year)', fontsize=14)


# 计算累积磨损
def cumulative_wear_rate(wear_rates):
    return np.cumsum(wear_rates) * dt

# 计算每一层楼梯的累积磨损
cumulative_wear_bottom = cumulative_wear_rate(wear_rates_bottom)
cumulative_wear_middle = cumulative_wear_rate(wear_rates_middle)
cumulative_wear_top = cumulative_wear_rate(wear_rates_top)

# 计算总累积磨损
total_cumulative_wear = cumulative_wear_bottom + cumulative_wear_middle + cumulative_wear_top

# 绘制每层楼梯的累积磨损图和总磨损
plt.figure(figsize=(14, 6))
plt.plot(Time, cumulative_wear_bottom, label="Bottom Layer Cumulative Wear", color='orange', linewidth=2)
plt.plot(Time, cumulative_wear_middle, label="Middle Layer Cumulative Wear", color='blue', linewidth=2)
plt.plot(Time, cumulative_wear_top, label="Top Layer Cumulative Wear", color='green', linewidth=2)
plt.plot(Time, total_cumulative_wear, label="Total Cumulative Wear", color='red', linewidth=3, linestyle='--')

plt.title('Cumulative Wear of Stairs Over Time', fontsize=16)
plt.xlabel('Year', fontsize=14)
plt.ylabel('Cumulative Wear (cm³)', fontsize=14)
plt.legend()
plt.grid(False)
plt.tight_layout()
plt.show()





# 添加图例
axes[2].legend()

# 绘制累计磨损图
plt.figure(figsize=(14, 6))

# 累积磨损图（底层、中层、顶层和总累积）
Z_bottom_cumulative = np.cumsum([wear_rate_layer(t, layer="bottom") for t in Time])
Z_middle_cumulative = np.cumsum([wear_rate_layer(t, layer="middle") for t in Time])
Z_top_cumulative = np.cumsum([wear_rate_layer(t, layer="top") for t in Time])
Z_total_cumulative = Z_bottom_cumulative + Z_middle_cumulative + Z_top_cumulative

# 绘制累积磨损曲线
plt.plot(mapped_years_weighted, Z_bottom_cumulative, label="Bottom Layer", color='orange')
plt.plot(mapped_years_weighted, Z_middle_cumulative, label="Middle Layer", color='blue')
plt.plot(mapped_years_weighted, Z_top_cumulative, label="Top Layer", color='green')
plt.plot(mapped_years_weighted, Z_total_cumulative, label="Total", color='red', linestyle='--')

plt.title('Cumulative Wear Over Time')
plt.xlabel('Year')
plt.ylabel('Cumulative Wear (Z)')
plt.grid(False)
plt.legend()

plt.tight_layout()
plt.show()








# 绘制年度游客人数图（绿色）
plt.figure(figsize=(50, 6))
plt.plot(Time, visitor_counts_updated, label='Annual Visitors (Updated)', color='green', linewidth=2)
plt.axvline(1372, color='gray', linestyle='--', label='1372: Completed')
plt.axvline(1838, color='gray', linestyle='--', label='1838: First Repair')
plt.axvline(1935, color='gray', linestyle='--', label='1935: WWII Impact')
plt.axvline(1990, color='gray', linestyle='--', label='1990: Closed for Repair')
plt.axvline(2001, color='gray', linestyle='--', label='2001: Reopened')

# 定义时间段
segments = [
    (1173, 1372),  # 建造时期
    (1372, 1838),  # 完工到首次修复
    (1838, 1934),  # 首次修复到二战
    (1934, 1990),  # 二战到关闭修复
    (1990, 2001),  # 修复到持续增长
    (2001, 2020),  # 高水平稳定
    (2020, 2022),  # 疫情影响
    (2022, 2025)   # 恢复增长
]


    
# 计算每段时间的长度
segment_lengths = [end - start for start, end in segments]

# 定义每个时间段的权重
segment_weights = [2,1,5,10,25,30,180,100]  # 权重可调节
normalized_weights = [w / sum(segment_weights) for w in segment_weights]  # 归一化权重

# 根据时间段长度和权重计算映射长度
segment_mapped_lengths = [length * weight for length, weight in zip(segment_lengths, normalized_weights)]
segment_positions_weighted = [0]  # 初始化横轴位置
for length in segment_mapped_lengths:
    segment_positions_weighted.append(segment_positions_weighted[-1] + length)


# 将年度数据重新映射到新的横轴比例
mapped_years_weighted = []
for t in Time:
    for i, (start, end) in enumerate(segments):
        if start <= t < end:
            segment_start = segment_positions_weighted[i]
            segment_end = segment_positions_weighted[i + 1]
            mapped_years_weighted.append(
                segment_start + (t - start) / (end - start) * (segment_end - segment_start)
            )
            break
# 确保最后一个点映射到最后的区间
if len(mapped_years_weighted) < len(visitor_counts_updated):
    mapped_years_weighted.append(segment_positions_weighted[-1])
# 绘制调整后时间比例的年度游客人数图（带时间段权重）
plt.figure(figsize=(14, 6))
plt.plot(mapped_years_weighted, visitor_counts_updated, label='Annual Visitors (Weighted Time Density)', color='green', linewidth=2)

# 添加分段竖线和横向标签
for i, (start, end) in enumerate(segments):
    plt.axvline(segment_positions_weighted[i], color='gray', linestyle='--')
    plt.text(
        (segment_positions_weighted[i] + segment_positions_weighted[i + 1]) / 2,
        max(visitor_counts_updated) * 0.8,
        f"{start}-{end}",
        fontsize=10,
        ha='center'
    )

# 设置横轴刻度和单位
plt.xticks(segment_positions_weighted, [f"{start}" for start, _ in segments] + ["2025"], rotation=0)
plt.xlabel("Year", fontsize=20)  # 横轴单位为年
plt.ylabel("Number of Visitors (Million Persons)", fontsize=12)  # 纵轴单位为人数

# 去除网格线
plt.grid(False)
plt.legend()
plt.tight_layout()
plt.show()


/var/folders/45/2y6n9gfj11ncxxgrf7wj2t8m0000gn/T/ipykernel_96201/3089856236.py:85: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  cumulative_visitors = np.trapz(visitors, dx=(t - 1173) / len(visitors))
/var/folders/45/2y6n9gfj11ncxxgrf7wj2t8m0000gn/T/ipykernel_96201/3089856236.py:135: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  cumulative_visitors = np.trapz(visitors, dx=(t - 1173) / len(visitors))
/var/folders/45/2y6n9gfj11ncxxgrf7wj2t8m0000gn/T/ipykernel_96201/3089856236.py:181: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  cumulative_visitors = np.trapz(visitors, dx=(t - 1173) / len(visitors))


RuntimeError: latex was not able to process the following string:
b'Friction Coefficient (\\u03bc)'

Here is the full command invocation and its output:

latex -interaction=nonstopmode --halt-on-error --output-directory=tmp23sek4qm af91f764a9f7cd8f7ba03083fa6e4888.tex

This is pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(./af91f764a9f7cd8f7ba03083fa6e4888.tex
LaTeX2e <2023-11-01> patch level 1
L3 programming layer <2024-02-20>
(/usr/local/texlive/2024/texmf-dist/tex/latex/base/article.cls
Document Class: article 2023/05/17 v1.4n Standard LaTeX document class
(/usr/local/texlive/2024/texmf-dist/tex/latex/base/size10.clo))
(/usr/local/texlive/2024/texmf-dist/tex/latex/type1cm/type1cm.sty)
(/usr/local/texlive/2024/texmf-dist/tex/latex/cm-super/type1ec.sty
(/usr/local/texlive/2024/texmf-dist/tex/latex/base/t1cmr.fd))
(/usr/local/texlive/2024/texmf-dist/tex/latex/base/inputenc.sty)
(/usr/local/texlive/

Error in callback <function _draw_all_if_interactive at 0x105c58d30> (for post_execute), with arguments args (),kwargs {}:


RuntimeError: latex was not able to process the following string:
b'Friction Coefficient (\\u03bc)'

Here is the full command invocation and its output:

latex -interaction=nonstopmode --halt-on-error --output-directory=tmplsdl3z8p af91f764a9f7cd8f7ba03083fa6e4888.tex

This is pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(./af91f764a9f7cd8f7ba03083fa6e4888.tex
LaTeX2e <2023-11-01> patch level 1
L3 programming layer <2024-02-20>
(/usr/local/texlive/2024/texmf-dist/tex/latex/base/article.cls
Document Class: article 2023/05/17 v1.4n Standard LaTeX document class
(/usr/local/texlive/2024/texmf-dist/tex/latex/base/size10.clo))
(/usr/local/texlive/2024/texmf-dist/tex/latex/type1cm/type1cm.sty)
(/usr/local/texlive/2024/texmf-dist/tex/latex/cm-super/type1ec.sty
(/usr/local/texlive/2024/texmf-dist/tex/latex/base/t1cmr.fd))
(/usr/local/texlive/2024/texmf-dist/tex/latex/base/inputenc.sty)
(/usr/local/texlive/

RuntimeError: latex was not able to process the following string:
b'Friction Coefficient (\\u03bc)'

Here is the full command invocation and its output:

latex -interaction=nonstopmode --halt-on-error --output-directory=tmp_ewungpd af91f764a9f7cd8f7ba03083fa6e4888.tex

This is pdfTeX, Version 3.141592653-2.6-1.40.26 (TeX Live 2024) (preloaded format=latex)
 restricted \write18 enabled.
entering extended mode
(./af91f764a9f7cd8f7ba03083fa6e4888.tex
LaTeX2e <2023-11-01> patch level 1
L3 programming layer <2024-02-20>
(/usr/local/texlive/2024/texmf-dist/tex/latex/base/article.cls
Document Class: article 2023/05/17 v1.4n Standard LaTeX document class
(/usr/local/texlive/2024/texmf-dist/tex/latex/base/size10.clo))
(/usr/local/texlive/2024/texmf-dist/tex/latex/type1cm/type1cm.sty)
(/usr/local/texlive/2024/texmf-dist/tex/latex/cm-super/type1ec.sty
(/usr/local/texlive/2024/texmf-dist/tex/latex/base/t1cmr.fd))
(/usr/local/texlive/2024/texmf-dist/tex/latex/base/inputenc.sty)
(/usr/local/texlive/

<Figure size 1400x1200 with 3 Axes>